In [1]:
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from keras.utils import array_to_img
import tensorflow_probability as tfp
from tensorflow.keras.layers import AveragePooling2D

sys.path.append("../models/GPA")
sys.path.append("../models/CD")
sys.path.append("../models/DS/")
sys.path.append("../")
from gpa import GPA
from cd import CD
from ds import DS

from utils import *
from simu_auxiliary import *

2025-10-15 15:39:39.989475: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-15 15:39:40.034128: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-15 15:39:40.738697: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


⚠️ `Note`:

- **TensorFlow 2.12.0** is compiled with **CUDA 11.8** and **cuDNN 8.6.0**.

- If your local CUDA/cuDNN version is lower (e.g., cuDNN 8.1.x), the following line may raise a "DNN library is not found" or "UnimplementedError" due to version mismatch:

> `avg_pool_2d_mean = AveragePooling2D(pool_size=(5, 5), strides=1, padding="same")`

- In that case, switch to Code Block (2) for an alternative CPU-based implementation.

In [3]:
# Code Block (1): Tensorflow GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

In [2]:
## Code Block (2): Tensorflow CPU
## this code will enforce tensorflow to use cpu
# tf.config.set_visible_devices([], 'GPU') 

## Hyper-parameters

In [4]:
T = 100                    # the number of replications
N_list = [100, 500, 1000]  # the number of images
p = 540                    # image height
q = 960                    # image width
M = p * q                  # the number of pixel locations
G = 500                    # the number of grid points
test_size = 100            # the number of test images
filter_size = 3            # filter size for kernel smoothing

## Distribution

In [5]:
# standard deviation & mean
sigma = 0.16218820445197027  # standard deviation
avg_pool_2d_mean = AveragePooling2D(pool_size=(5, 5), strides=1, padding="same")
mean = np.load('./mean-540.npy').reshape([1, p, q, 1])  # mean
mean = tf.reshape(avg_pool_2d_mean(mean), (1, p, q))    # smoothed mean

In [6]:
# grid points: randomly generated from U[0, 1]
np.random.seed(0)
tf.random.set_seed(0)
tick_list = np.random.uniform(size=G)
grid_point = tf.concat([tf.ones([1, p, q]) * tick for tick in tick_list], axis=0)

# oracle distribution
tfd = tfp.distributions
dist = tfd.TruncatedNormal(loc=mean, scale=sigma, low=[0.], high=[1.])

## Run Experiments

- Abbreviations:
    - CD: Classical nonparametric Density
    - DS: Doubly Smoothed
    - GPA: Grid Point Approximation

In [7]:
experiment_stats = {"N": [],            # sample size
                    "t": [],            # id of the replication
                    "CD_mse": [],       # MSE of the CD method
                    "DS_mse": [],       # MSE of the DS method
                    "GPA_CD_mse": [],   # MSE of the GPA-CD method
                    "GPA_DS_mse": [],   # MSE of the GPA-DS method
                    "CD_time": [],      # computation time of the CD method
                    "DS_time": [],      # computation time of the DS method
                    "GPA_CD_time": [],  # computation time of the GPA-CD method
                    "GPA_DS_time": []}  # computation time of the GPA-DS method

In [ ]:
# == Main experiment loop ==
# -- Assumption: `test_imgs` and `test_orcs` are precomputed outside (above chunk),
# -- so every method, N (sample size), and t (replication index) is evaluated
# -- on the SAME test set for fair comparison.
for N in N_list:
    print(f"====================== N={N} ======================")
    # -- Change path as you needed
    path = f'../../../[simu]train_img(N={N})/'  # Directory to save N simulated images
    bandwidth, bandwidth_star = compute_optimal_bandwidths(N, M, sigma)  # (rule-of-thumb) bandwidths
    location_filter = get_location_filter(p, q, bandwidth, filter_size)
    
    # == Replicate loop (T times) ==
    # -- Different training sets per replicate
    for t in range(T):
        print(f"======== N={N}: The {t}-th replication ========")
        # -- Set seeds so all methods within the same replicate share training randomness
        seed = N + t
        tf.random.set_seed(seed)
        np.random.seed(seed)
        
        # -- Initialize cumulative metrics
        CD_mse = 0.0
        DS_mse = 0.0
        GPA_CD_mse = 0.0
        GPA_DS_mse = 0.0
        CD_time = 0.0
        DS_time = 0.0
        GPA_CD_time = 0.0
        GPA_DS_time = 0.0
        
        # -- Generate the training set for this replicate
        train_list = generate_simulate_data(path, N, mean, sigma)
        
        # -- Build models once per replicate
        CD_model = CD(p, q, train_list)
        
        ### --- this will fit DS model from raw
        # DS_model = DS(p, q, train_list) 
        
        ### --- this will fit GPA-CD model from raw
        # GPA_CD_model = GPA(G, p, q, train_list, second_smooth=False, gpa_matrix=None,
        #                    grid_point=grid_point, h=bandwidth, h_star=bandwidth_star, filter_size=None)
        
        ### --- this will fit GPA-DS model from raw
        # GPA_DS_model = GPA(G, p, q, train_list, second_smooth=True, gpa_matrix=None,
        #                    grid_point=grid_point, h=bandwidth, h_star=bandwidth_star, filter_size=filter_size)
        
        ### --- the below computation of GPA-DS is computed based on GPA-CD, which will save time
        ### --- this is because GPA_DS_model = GPA(xxx) will rerun the GPA-CD computation and then doubly smooth
        ### --- thus, to save time, we directly compute GPA-CD matrix and then doubly smooth it outside the models
        GPA_CD_matrix = compute_CD_matrix(path, N, G, p, q, bandwidth, grid_point)
        Omega1 = tf.nn.depthwise_conv2d(tf.reshape(GPA_CD_matrix, [G, p, q, 1]), 
                                        location_filter, strides=[1, 1, 1, 1], padding='SAME')
        Omega2 = tf.reduce_sum(location_filter)
        GPA_DS_matrix = Omega1 / Omega2
        GPA_CD_matrix = tf.squeeze(GPA_CD_matrix)
        GPA_DS_matrix = tf.squeeze(GPA_DS_matrix)
        ### --- Fit the model based on pre-computed GPA matrix
        GPA_CD_model = GPA(G, p, q, train_list, second_smooth=False, gpa_matrix=GPA_CD_matrix,
                           grid_point=grid_point, h=bandwidth, h_star=bandwidth_star, filter_size=None)
        GPA_DS_model = GPA(G, p, q, train_list, second_smooth=True, gpa_matrix=GPA_DS_matrix, 
                           grid_point=grid_point, h=bandwidth, h_star=bandwidth_star, filter_size=filter_size)
        
        # == Evaluate four methods on the test set ==
        for test_i in range(test_size):
            # -- The test image and its oracle density
            test_img = tf.ones([p, q]) * np.random.uniform()
            test_orc = tf.squeeze(dist.prob(test_img))
            
            # -- CD 
            t1 = time.time()
            CD_test = CD_model.CD_estimation(test_img, bandwidth)
            t2 = time.time()
            CD_time += t2 - t1
            
            # -- DS 
            t3 = time.time()
            # DS_test = DS_model.DS_estimation(test_img, bandwidth, size=filter_size)
            CD_test = tf.reshape(CD_test, (1, p, q))
            DS_test = compute_DS_matrix(CD_test, location_filter)
            t4 = time.time()
            DS_time += t2 - t1 + t4 - t3
            
            # -- GPA-CD 
            t1 = time.time()
            GPA_CD_test = GPA_CD_model.compute_density(test_img)
            t2 = time.time()
            GPA_CD_time += t2 - t1
            
            # -- GPA-DS 
            t1 = time.time()
            GPA_DS_test = GPA_DS_model.compute_density(test_img)
            t2 = time.time()
            GPA_DS_time += t2 - t1

            # -- Accumulate MSE; average over test_size after the loop
            CD_mse     += MSE(test_orc, tf.squeeze(CD_test))
            DS_mse     += MSE(test_orc, tf.squeeze(DS_test)) 
            GPA_CD_mse += MSE(test_orc, tf.squeeze(GPA_CD_test))
            GPA_DS_mse += MSE(test_orc, tf.squeeze(GPA_DS_test)) 
        
        # -- Average metrics for this replicate (divide by test_size)
        experiment_stats["N"].append(N)
        experiment_stats["t"].append(t)
        experiment_stats["CD_mse"].append(CD_mse.numpy() / test_size)
        experiment_stats["DS_mse"].append(DS_mse.numpy() / test_size)
        experiment_stats["GPA_CD_mse"].append(GPA_CD_mse.numpy() / test_size)
        experiment_stats["GPA_DS_mse"].append(GPA_DS_mse.numpy() / test_size)
        experiment_stats["CD_time"].append(CD_time / test_size)
        experiment_stats["DS_time"].append(DS_time / test_size)
        experiment_stats["GPA_CD_time"].append(GPA_CD_time / test_size)
        experiment_stats["GPA_DS_time"].append(GPA_DS_time / test_size)
        
        # -- Timely record the results
        experiment_stats_csv = pd.DataFrame(experiment_stats)
        experiment_stats_csv.to_csv(f'./results/new_simulation.csv', index=False)